In [2]:
import pandas as pd
import numpy as np

t = pd.read_csv ('train.csv')
t.head ()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
t.describe ()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
t.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Процент выживаемости у каждого класса пассажиров (Pclass)

In [69]:
sur = t.groupby(['Pclass'], as_index=False)[['Survived']].sum()
sur

,Pclass,PassengerId
0,1,216
1,2,184
2,3,491


In [70]:
Pid = t.groupby(['Pclass'], as_index=False)[['PassengerId']].count()
Pid

,Pclass,PassengerId
0,1,216
1,2,184
2,3,491


In [136]:
Percentage = pd.DataFrame ({'Pclass': sur['Pclass'], 'Survived': sur['Survived'], 'total': Pid['PassengerId']})
Percentage

,Pclass,Survived,total
0,1,136,216
1,2,87,184
2,3,119,491


In [72]:
Percentage['Percentage'] = Percentage['Survived']/Percentage['total']*100
Percentage

,Pclass,Survived,total,Percentage
0,1,136,216,62.962963
1,2,87,184,47.282609
2,3,119,491,24.236253


# Выведите самое популярное мужское и самое популярное женское имя на корабле

In [12]:
#Мужское имя следует за Mr.
#Делим строчку по точке и берем соответсвующий элемент

t_male = t.drop(t[t['Sex'] == 'female'].index)
t_male['First_Name'] = t_male['Name'].str.split(' ').str[2]

print (t_male['First_Name'])

name_counts = t_male['First_Name'].value_counts()

print(f"Самое популярное мужское имя: {name_counts.head(1).index.values[0]}")

0           Owen
4        William
5          James
6        Timothy
7          Gosta
         ...    
883    Frederick
884        Henry
886       Juozas
889         Karl
890      Patrick
Name: First_Name, Length: 577, dtype: object
Самое популярное мужское имя: William


In [14]:
#Женское имя требует три фильтра: Mrs. - первое в скобках или Miss. - сразу за точкой

t_female = t.drop(t[t['Sex'] == 'male'].index)
#t_female['First_Name'] = t_female['Name'].str.split('.').str[1]


#print (t_female['First_Name'])

def first_name_female (t_female):
    first = t_female['Name'].str.extract(r"Mrs\.[^(]*\((\w+)", expand=False)
    first.loc[first.isna()] = t_female['Name'].str.extract(r"\.\s+(\w+)", expand=False)
    return first
female_names = first_name_female (t_female)
print (female_names)


name_counts = female_names.value_counts()

print(f"Самое популярное женское имя: {name_counts.head(1).index.values[0]}")

1       Florence
2          Laina
3           Lily
8      Elisabeth
9          Adele
         ...    
880      Imanita
882        Gerda
885     Margaret
887     Margaret
888    Catherine
Name: Name, Length: 314, dtype: object
Самое популярное женское имя: Anna


# Выведите самое популярное мужское и самое популярное женское имя на корабле в каждом классе

In [249]:
#Мужское имя

t_male_all = t.drop(t[t['Sex'] == 'female'].index)
t_male_all['First_Name'] = t_male_all['Name'].str.split('.').str[1].str.split(" ").str[1]
t_male_1_3 = t_male_all.drop(t_male_all[t_male_all['Pclass'] == 2].index)
t_male_1 = t_male_1_3.drop(t_male_1_3[t_male_1_3['Pclass'] == 3].index)
t_male_2_3 = t_male_all.drop(t_male_all[t_male_all['Pclass'] == 1].index)
t_male_2 = t_male_2_3.drop(t_male_2_3[t_male_2_3['Pclass'] == 1].index)
t_male_3 = t_male_1_3.drop(t_male_1_3[t_male_1_3['Pclass'] == 1].index)

name_counts_1 = t_male_1['First_Name'].value_counts()
print(f"Самое популярное мужское имя в первом классе: {name_counts_1.head(1).index.values[0]}")

name_counts_2 = t_male_2['First_Name'].value_counts()
print(f"Самое популярное мужское имя во втором классе: {name_counts_2.head(1).index.values[0]}")

name_counts_3 = t_male_3['First_Name'].value_counts()
print(f"Самое популярное мужское имя в третьем классе: {name_counts_3.head(1).index.values[0]}")

Самое популярное мужское имя в первом классе: William
Самое популярное мужское имя во втором классе: William
Самое популярное мужское имя в третьем классе: William


In [254]:
#Женское имя

t_female = t.drop(t[t['Sex'] == 'male'].index)
t_female_1_3 = t_female.drop(t_female[t_female['Pclass'] == 2].index)
t_female_1 = t_female_1_3.drop(t_female_1_3[t_female_1_3['Pclass'] == 3].index)
t_female_2_3 = t_female.drop(t_female[t_female['Pclass'] == 1].index)
t_female_2 = t_female_2_3.drop(t_female_2_3[t_female_2_3['Pclass'] == 1].index)
t_female_3 = t_female_1_3.drop(t_female_1_3[t_female_1_3['Pclass'] == 1].index)


def first_name_female_1 (t_female_1):
    first_1 = t_female_1['Name'].str.extract(r"Mrs\.[^(]*\((\w+)", expand=False)
    first_1.loc[first_1.isna()] = t_female_1['Name'].str.extract(r"\.\s+(\w+)", expand=False)
    return first_1
female_names_1 = first_name_female_1 (t_female_1)

name_counts_1 = female_names_1.value_counts()
print(f"Самое популярное женское имя в первом классе: {name_counts_1.head(1).index.values[0]}")


def first_name_female_2 (t_female_2):
    first_2 = t_female_2['Name'].str.extract(r"Mrs\.[^(]*\((\w+)", expand=False)
    first_2.loc[first_2.isna()] = t_female_2['Name'].str.extract(r"\.\s+(\w+)", expand=False)
    return first_2
female_names_2 = first_name_female_2 (t_female_2)

name_counts_2 = female_names_2.value_counts()
print(f"Самое популярное женское имя во втором классе: {name_counts_2.head(1).index.values[0]}")

def first_name_female_3 (t_female_3):
    first_3 = t_female_3['Name'].str.extract(r"Mrs\.[^(]*\((\w+)", expand=False)
    first_3.loc[first_3.isna()] = t_female_3['Name'].str.extract(r"\.\s+(\w+)", expand=False)
    return first_3
female_names_3 = first_name_female_3 (t_female_3)

name_counts_3 = female_names_3.value_counts()
print(f"Самое популярное женское имя в третьем классе: {name_counts_3.head(1).index.values[0]}")


Самое популярное женское имя в первом классе: Elizabeth
Самое популярное женское имя во втором классе: Anna
Самое популярное женское имя в третьем классе: Anna


# Выведите часть таблицы с пассажирами, возраст которых больше 44 лет

In [255]:
age_over_44 = t[t['Age'] > 44]
age_over_44

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,First_Name
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Timoth
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S,Elizabeth
15,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,NaN,S,(Mar
33,34,0,2,"Wheadon, Mr. Edward H",male,66.0,0,0,C.A. 24579,10.5000,NaN,S,Edwar
52,53,1,1,"Harper, Mrs. Henry Sleeper (Myna Haxtun)",female,49.0,1,0,PC 17572,76.7292,D33,C,Henr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,858,1,1,"Daly, Mr. Peter Denis",male,51.0,0,0,113055,26.5500,E17,S,Pete
862,863,1,1,"Swift, Mrs. Frederick Joel (Margaret Welles Ba...",female,48.0,0,0,17466,25.9292,D17,S,Frederic
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S,Richar
873,874,0,3,"Vander Cruyssen, Mr. Victor",male,47.0,0,0,345765,9.0000,NaN,S,Mr


# Выведите часть таблицы с пассажирами, возраст которых меньше 44 лет и которые мужского пола

In [258]:
age_under_44 = t[t['Age'] < 44]
t_male_age_under_44 = age_under_44.drop(age_under_44[age_under_44['Sex'] == 'female'].index)
t_male_age_under_44

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,First_Name
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.250,NaN,S,Owe
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.050,NaN,S,Willia
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.075,NaN,S,Gost
12,13,0,3,"Saundercock, Mr. William Henry",male,20.0,0,0,A/5. 2151,8.050,NaN,S,Willia
13,14,0,3,"Andersson, Mr. Anders Johan",male,39.0,1,5,347082,31.275,NaN,S,Ander
...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,884,0,2,"Banfield, Mr. Frederick James",male,28.0,0,0,C.A./SOTON 34068,10.500,NaN,S,Frederic
884,885,0,3,"Sutehall, Mr. Henry Jr",male,25.0,0,0,SOTON/OQ 392076,7.050,NaN,S,Henr
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.000,NaN,S,Juozas
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.000,C148,C,Kar



# Выведите количества n-местных кабин (в которых было 2, 3, 4, ... человека)

In [19]:
t.groupby('Cabin').size().value_counts()[1:]

,count
2,38
3,5
4,3
